Make sure you update the **STUDENT_TOKEN**, and then run this block. You should use FIRSTNAME-SURNAME(S) as the TOKEN.

The code in this block can be safely ignored.

In [1]:
#Update your token
STUDENT_TOKEN = 'EUGENIO MARCHIORI'

## ignore this code, just used for submission
import requests
import pprint
import json
import random
from copy import copy, deepcopy

def get_problem(problem_id):
  r = requests.get('https://emarchiori.eu.pythonanywhere.com/get-problem?TOKEN=%s&problem=%s' % (STUDENT_TOKEN, problem_id))
  if not r.status_code == 200:
    print('\033[91m' + str(r.status_code))
  return r.json()

def submit_answer(problem_id, answer):
  r = requests.get('https://emarchiori.eu.pythonanywhere.com/submit-answer?TOKEN=%s&problem=%s' % (STUDENT_TOKEN, problem_id), json = answer)
  if not r.status_code == 200:
    print('\033[91m' + str(r.status_code))
  result = r.json()['result']
  if result == 'PASSED':
    print('\033[92m' + result)
  else:
    print('\033[91m' + result)


In [2]:
# 0 represents unassinged values

# Helper functions for Sudoku: Feel free to use or implement your own

def valid_row(sudoku_board, row):
  assigned = []
  for value in sudoku_board[row]:
    if value != 0:
      if value in assigned:
        return False
      else:
        assigned.append(value)
  return True

def valid_column(sudoku_board, column):
  assigned = []
  for row in sudoku_board:
    value = row[column]
    if value != 0:
      if value in assigned:
        return False
      else:
        assigned.append(value)
  return True

# Quadrants are numbered from top left to bottom right
def valid_quadrant(sudoku_board, quadrant):
  starting_row = (quadrant // 3) * 3
  starting_column = (quadrant % 3) * 3
  assigned = []
  for row in sudoku_board[starting_row : starting_row + 3]:
    for value in row[starting_column : starting_column + 3]:
      if value in assigned:
        return False
      assigned.append(value)
  return True

# Check if a solution is complete (i.e. there are no 0)
def complete_solution(sudoku_board):
  for row in sudoku_board:
    for value in row:
      if value == 0:
        return False
  return True

# Check if a solution is valid (i.e. respects all constraints)
def valid_solution(sudoku_board):
  for i in range(9):
    if not valid_row(sudoku_board, i):
      return False
    if not valid_column(sudoku_board, i):
      return False
    if not valid_quadrant(sudoku_board, i):
      return False
  return True

# Checks that the solution respects all constraints (including values set in the problem) and is complete
def check_solution(puzzle, solution):
  for row in range(9):
    for col in range(9):
      if puzzle[row][col] != 0 and puzzle[row][col] != solution[row][col]:
        return 'Solution is not a valid, changes a fixed value in the puzzle'
  if not valid_solution(solution):
    return 'Solution does not respect the constraints'
  if not complete_solution(solution):
    return 'Solution is only partial'

def pretty_print(sudoku_board):
  for row in sudoku_board:
    print(row)

# Helper functions to get and set the values from the server
def sudoku_string_to_matrix(sudoku):
  sudoku_board = []
  for i in range(9):
    starting_row = i * 9
    row = []
    for i in range(9):
      row.append(int(sudoku[starting_row + i]))
    sudoku_board.append(row)
  return sudoku_board

def sudoku_matrix_to_string(sudoku_board):
  return ''.join(map(lambda row : ''.join(map(str, row)), sudoku_board))


In [13]:
# Random Sudoku solver: assigns random values to all unassigned cells.

def random_solver(sudoku_board):
  while True:
    board = deepcopy(sudoku_board)
    for row in range(9):
      for col in range(9):
        if board[row][col] == 0:
          board[row][col] = random.randint(1, 9)
    if valid_solution(board):
      return board
  return sudoku_board

def backtracking(board):
  return backtrack(board, deepcopy(board))

def backtrack(board, assignment):
  if complete_solution(assignment):
    return assignment
  var = (0,0)
  for i in range(9):
    for j in range(9):
      if assignment[i][j] == 0:
        var = (i,j)
        break

  for value in range(1,10):
    assignment[var[0]][var[1]] = value
    if valid_solution(assignment):
       result = backtrack(board, assignment)
       if result is not None: return result
    assignment[var[0]][var[1]] = 0

  return None


In [19]:
# Sudoku solver using the "Random solver" method

problem = get_problem('easy-sudoku') # try also hard-sudoku

pprint.pprint(problem)

sudoku_board = sudoku_string_to_matrix(problem['sudoku'])

# Un-comment these lines to override the initial state provided by the server
# Note: This is good during de-bugging, to make sure we don't get changing problems, and can compare different runs
# super easy
# sudoku_board = [[8, 2, 0, 6, 3, 7, 9, 4, 1],[1, 3, 4, 0, 9, 2, 5, 6, 7],[9, 6, 7, 1, 4, 5, 8, 2, 3],[2, 1, 8, 3, 5, 9, 4, 7, 6],[3, 4, 9, 2, 7, 6, 1, 8, 5],[5, 7, 6, 4, 8, 1, 2, 3, 9],[4, 8, 1, 9, 6, 0, 7, 5, 2],[6, 5, 2, 7, 1, 8, 3, 9, 4],[7, 9, 3, 5, 0, 4, 6, 1, 8]]
#sudoku_board = [[8, 2, 5, 6, 3, 7, 0, 4, 1],[1, 0, 4, 8, 9, 2, 5, 6, 7],[9, 6, 7, 1, 4, 5, 8, 2, 3],[2, 1, 8, 3, 5, 9, 4, 7, 6],[3, 4, 9, 2, 7, 6, 1, 8, 5],[5, 7, 6, 4, 8, 1, 2, 3, 9],[4, 8, 1, 9, 6, 3, 7, 5, 2],[6, 5, 2, 7, 1, 8, 3, 9, 4],[7, 9, 3, 5, 0, 4, 6, 1, 8]]

print('Input puzzle')
pretty_print(sudoku_board)

# This solution uses the random solver (which might never find a solution even in easy boards).
solution = backtracking(sudoku_board)

# We print the solution in a way to make it easy to debug and check
print('Solution')
pretty_print(solution)
check_solution(sudoku_board, solution)

answer = {
    'sudoku': sudoku_matrix_to_string(solution),
    'id': problem['id']
}

# The answer is submitted to the server, this is just to check the format (it does not check the solution itself)
# Note: We are doing this just to get used to it in following exercises mostly
submit_answer('sudoku', answer)

{'id': 3,
 'sudoku': '000016020630405107500000340100800405004000260056004000908000002245790601067152980'}
Input puzzle
[0, 0, 0, 0, 1, 6, 0, 2, 0]
[6, 3, 0, 4, 0, 5, 1, 0, 7]
[5, 0, 0, 0, 0, 0, 3, 4, 0]
[1, 0, 0, 8, 0, 0, 4, 0, 5]
[0, 0, 4, 0, 0, 0, 2, 6, 0]
[0, 5, 6, 0, 0, 4, 0, 0, 0]
[9, 0, 8, 0, 0, 0, 0, 0, 2]
[2, 4, 5, 7, 9, 0, 6, 0, 1]
[0, 6, 7, 1, 5, 2, 9, 8, 0]
Solution


TypeError: ignored